In [ ]:
!pip install pycaret

In [ ]:
import numpy as np

from pycaret.classification import (
    setup,
    compare_models,
    create_model,
    tune_model,
    plot_model,
    evaluate_model,
    predict_model,
    finalize_model,
    save_model,
    load_model
)
from pycaret.datasets import get_data

# PyCaret para Classificação

Neste exercício, você vai escolher outra base de dados do Pycaret para repetir a pipeline de treinamento realizada em aula.

## Base de dados

Recupere todas as bases de dados disponibilizadas pelo Pycaret.

In [ ]:
all_bases = get_data('index')

Desta vez, vamos utilizar o dataset `questions`, que é um dataset de classificação com mais de duas classes.

Recupere este dataset.

In [ ]:
dataset = get_data('questions')

Não existe documentação sobre este dataset, mas podemos inferir pelas colunas que, dada a frequência das categorias 1, 2 e 3, se busca inferir a caegoria da próxima pergunta (`Next_Question`).

Verifique quantas instâncias e quantos atributos este dataset possui.

In [ ]:
dataset.shape()

Verifique as categorias (coluna `Next_Question`) e sua respectiva contagem.

In [ ]:
np.unique(dataset['Next_Question'], return_counts=True)

Gere uma base de treinamento correspondente a 90% dos dados totais.

In [ ]:
base_train = dataset.sample(frac=0.9, random_state=1)
base_train.shape

Gere a base de testes, correspondente aos dados totais que não estão presentes na base de treinamento.

In [ ]:
base_test = dataset.drop(base_train.index)
base_test.shape

## Configuração do ambiente PyCaret

Gere um experimento, utilizando a base de treinamento.

In [ ]:
experiment = setup(data=base_train, target='Next_Question', session_id=1)

## Treinamento de vários modelos com validação cruzada

Execute uma comparação de modelos, organizando os resultandos pela coluna `Accuracy`. Solicite 10 folds.

In [ ]:
models = compare_models(fold=10, sort='Accuracy')

## Tuning dos modelos

Pelos resultados observados, você pode ver que alguns modelos com maior acurácia têm métrica AUC igual a 0. Então, crie um modelo do tipo Naive Bayes, já que é o primeiro modelo na lista com métrica AUC alta.

In [ ]:
naive_bayes = create_model('nb')

Exiba os parâmetros deste modelo.

In [ ]:
print(naive_bayes)

Execute o tuning do modelo.

In [ ]:
naive_bayes_tuning = tune_model(naive_bayes)

Exiba os parâmetros do modelo ajustado.

In [ ]:
print(naive_bayes_tuning)

Agora crie um modelo do tipo Light Gradient Boosting Machine.

In [ ]:
lightgbm = create_model('lightgbm')

Exiba os parâmetros do modelo.

In [ ]:
print(lightgbm)

Faça o tuning do modelo, investindo o número de estimadores (`n_estimators`) no intervalo 50 a 100, com passos de 10.

In [ ]:
lightgbm_tuning = tune_model(lightgbm, custom_grid = {'n_estimators': np.arange(50, 101, 10)})

Exiba os parâmetros do modelo ajustado.

In [ ]:
print(lightgbm_tuning)

## Gráficos

Exiba um gráfico da matriz de confusão do modelo Naive Bayes ajustado por tuning de parâmetros.

In [ ]:
plot_model(naive_bayes_tuning, plot='confusion_matrix')

Agora exiba um gráfico do reporte por classe.

In [ ]:
plot_model(naive_bayes_tuning, plot='class_report')

Exiba um gráfico do tipo fronteira (boundary).

In [ ]:
plot_model(naive_bayes_tuning, plot='boundary')

Exiba um gráfico com os erros do algoritmo.

In [ ]:
plot_model(naive_bayes_tuning, plot='error')

Exiba a avaliação do modelo.

In [ ]:
evaluate_model(naive_bayes_tuning)

## Previsões com o modelo

Execute as previsões do modelo.

In [ ]:
predict_model(naive_bayes_tuning);

Crie uma versão final do modelo ajustado por tuning de hiperparâmetros.

In [ ]:
naive_bayes_final = finalize_model(naive_bayes_tuning)
print(naive_bayes_final)

Salve este modelo.

In [ ]:
save_model(naive_bayes_final, 'naive_bayes')

Recarregue o modelo salvo.

In [ ]:
naive_bayes_loaded = load_model('/content/naive_bayes')

Faça previsões com o modelo recarregado na base de testes.

In [ ]:
prevision = predict_model(naive_bayes_loaded, data=base_test)
prevision